In [8]:
%pip install torch torchvision
%pip install trimesh numpy scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Import necessary libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import json
import trimesh
import os
import random
import shutil

Read data

In [11]:
data = pd.read_csv('/home/aalab/Desktop/tripoFT/metadata.csv')

In [12]:
print("The first few rows of the DataFrame:")
print(data.head())

print("\nData info:")
print(data.info())

print("\nData columns:")
print(data.columns)

print("\nData described:")
print(data.describe())

The first few rows of the DataFrame:
                                 filename      size  \
0  cadbury_dairy_milk_chocolate_piece.glb    152796   
1          bonnie_melted_chocolate_ar.glb   8285860   
2              chocobar_ice_cream (1).glb   4499300   
3                           ice_cream.glb  12444648   
4                   chocolate_truffle.glb   2084100   

                                         object_path  
0  /home/aalab/Desktop/tripoFT/Dataset/cadbury_da...  
1  /home/aalab/Desktop/tripoFT/Dataset/bonnie_mel...  
2  /home/aalab/Desktop/tripoFT/Dataset/chocobar_i...  
3  /home/aalab/Desktop/tripoFT/Dataset/ice_cream.glb  
4  /home/aalab/Desktop/tripoFT/Dataset/chocolate_...  

Data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   filename     145 non-null    object
 1   size         145 non-null    int64 
 2   object_path  145 

Clean Data

In [13]:
data.drop_duplicates(inplace=True)

# Drop rows with missing values in specific columns
columns_with_missing_values = ['filename', 'size', 'object_path']
data.dropna(subset=columns_with_missing_values, inplace=True)

# Convert columns to appropriate data types
data['filename'] = data['filename'].astype(str)
data['size'] = data['size'].astype(str)
data['object_path'] = data['object_path'].astype(str)

# Print cleaned DataFrame
print(data)

                                   filename      size  \
0    cadbury_dairy_milk_chocolate_piece.glb    152796   
1            bonnie_melted_chocolate_ar.glb   8285860   
2                chocobar_ice_cream (1).glb   4499300   
3                             ice_cream.glb  12444648   
4                     chocolate_truffle.glb   2084100   
..                                      ...       ...   
140                     strawberry_cake.glb  25159596   
141                jiggly_bunny_pudding.glb   1026416   
142            kitchen_sink_fiesta_5149.glb  26103980   
143   easter_egg_2024_marbled_chocolate.glb   3743544   
144              paleta_payaso_lollypop.glb    439920   

                                           object_path  
0    /home/aalab/Desktop/tripoFT/Dataset/cadbury_da...  
1    /home/aalab/Desktop/tripoFT/Dataset/bonnie_mel...  
2    /home/aalab/Desktop/tripoFT/Dataset/chocobar_i...  
3    /home/aalab/Desktop/tripoFT/Dataset/ice_cream.glb  
4    /home/aalab/Desktop/tripo

Convert 3D Objects into a Usable Format that our model can work with, such as voxel grids. 

In [14]:
input_directory = '/home/aalab/Desktop/tripoFT/Dataset'
output_directory = '/home/aalab/Desktop/tripoFT/tensors'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Function to process and save a single .glb file
def process_glb_file(file_path, output_dir):
    # Load the .glb file
    scene = trimesh.load(file_path)
    
    # Check if the loaded object is a Scene or a single Mesh
    if isinstance(scene, trimesh.Scene):
        meshes = scene.geometry.values()
    else:
        meshes = [scene]

    for i, mesh in enumerate(meshes):
        # Extract vertices and faces
        vertices = mesh.vertices
        faces = mesh.faces
        
        # Convert to tensors
        vertices_tensor = torch.tensor(vertices, dtype=torch.float32)
        faces_tensor = torch.tensor(faces, dtype=torch.int64)
        
        # Create file base name without extension and append index for multiple meshes
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        mesh_suffix = f'_{i}' if len(meshes) > 1 else ''
        
        # Save tensors to files
        torch.save(vertices_tensor, os.path.join(output_dir, f'{base_name}{mesh_suffix}_vertices.pt'))
        torch.save(faces_tensor, os.path.join(output_dir, f'{base_name}{mesh_suffix}_faces.pt'))

# Iterate over all .glb files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.glb'):
        file_path = os.path.join(input_directory, filename)
        print(f'Processing {file_path}')
        process_glb_file(file_path, output_directory)

print('Conversion complete.')

Processing /home/aalab/Desktop/tripoFT/Dataset/cadbury_dairy_milk_chocolate_piece.glb
Processing /home/aalab/Desktop/tripoFT/Dataset/bonnie_melted_chocolate_ar.glb
Processing /home/aalab/Desktop/tripoFT/Dataset/chocobar_ice_cream (1).glb
Processing /home/aalab/Desktop/tripoFT/Dataset/ice_cream.glb
Processing /home/aalab/Desktop/tripoFT/Dataset/chocolate_truffle.glb
Processing /home/aalab/Desktop/tripoFT/Dataset/oreo.glb
Processing /home/aalab/Desktop/tripoFT/Dataset/heart_cake.glb
Processing /home/aalab/Desktop/tripoFT/Dataset/candy_vending_machine.glb
Processing /home/aalab/Desktop/tripoFT/Dataset/gummy_bear_bull.glb
Processing /home/aalab/Desktop/tripoFT/Dataset/valentines_heart.glb
Processing /home/aalab/Desktop/tripoFT/Dataset/beardpapas_smores_puff__scan.glb
Processing /home/aalab/Desktop/tripoFT/Dataset/doughnut_pack.glb
Processing /home/aalab/Desktop/tripoFT/Dataset/cc0_-_candies.glb
Processing /home/aalab/Desktop/tripoFT/Dataset/biscuit.glb
Processing /home/aalab/Desktop/tripoF

Split the data

In [7]:
tensor_directory = '/home/aalab/Desktop/tripoFT/tensors'

# Directory to save the splits
train_dir = '/home/aalab/Desktop/tripoFT/split_data/train'
val_dir = '/home/aalab/Desktop/tripoFT/split_data/val'
test_dir = '/home/aalab/Desktop/tripoFT/split_data/test'

# # Create directories if they don't exist
for directory in [train_dir, val_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)

# Set the seed for reproducibility
random.seed(42)

# List of voxel tensor files
tensor_files = os.listdir(tensor_directory)
random.shuffle(tensor_files)

# Define the split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculate the number of files for each split
num_files = len(tensor_files)
num_train = int(train_ratio * num_files)
num_val = int(val_ratio * num_files)

# Split the files
train_files = tensor_files[:num_train]
val_files = tensor_files[num_train:num_train + num_val]
test_files = tensor_files[num_train + num_val:]

# Move files to respective directories
for file in train_files:
    shutil.move(os.path.join(tensor_directory, file), os.path.join(train_dir, file))

for file in val_files:
    shutil.move(os.path.join(tensor_directory, file), os.path.join(val_dir, file))

for file in test_files:
    shutil.move(os.path.join(tensor_directory, file), os.path.join(test_dir, file))

print("Data split into train, validation, and test sets.")
print(num_files, num_train, num_val)

Data split into train, validation, and test sets.
6432 4502 964


Load the Data

In [8]:
from torch.utils.data import Dataset, DataLoader

class CombinedTensorDataset(Dataset):
    def __init__(self, data_dir):
        """
        Args:
            data_dir (str): Directory with all the combined tensor files.
        """
        self.tensor_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.pt')])
        self.data_dir = data_dir

    def __len__(self):
        """Returns the total number of samples."""
        return len(self.tensor_files)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the sample to retrieve.
            
        Returns:
            tuple: (vertices, faces) where vertices is the tensor of vertices and faces is the tensor of faces.
        """
        tensor_path = os.path.join(self.data_dir, self.tensor_files[idx])
        
        # Load the combined tensor file
        combined_tensor = torch.load(tensor_path)
        
        # Assuming the combined tensor is a tuple (vertices, faces)
        vertices, faces = combined_tensor
        
        return vertices, faces

# Directories containing your combined tensor files
train_data_dir = '/home/aalab/Desktop/tripoFT/split_data/train'
val_data_dir = '/home/aalab/Desktop/tripoFT/split_data/val'
test_data_dir = '/home/aalab/Desktop/tripoFT/split_data/test'

# Create dataset instances
train_dataset = CombinedTensorDataset(train_data_dir)
val_dataset = CombinedTensorDataset(val_data_dir)
test_dataset = CombinedTensorDataset(test_data_dir)

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Get the TripoSR Model directory

In [1]:
import sys
import os

# Add the TripoSR directory to the Python path
sys.path.append(os.path.abspath('TripoSR'))

In [10]:
# Replace 'path/to/requirements.txt' with the actual path to your requirements.txt file
%pip install -r /home/aalab/Desktop/tripoFT/TripoSR/requirements.txt

  Cloning https://github.com/tatsy/torchmcubes.git to /tmp/pip-req-build-2poytg1o
  Running command git clone --filter=blob:none --quiet https://github.com/tatsy/torchmcubes.git /tmp/pip-req-build-2poytg1o
  Resolved https://github.com/tatsy/torchmcubes.git to commit 3aef8afa5f21b113afc4f4ea148baee850cbd472
  Preparing metadata (setup.py) ... done
  Using cached Pillow-10.1.0-cp310-cp310-manylinux_2_28_x86_64.whl (3.6 MB)
  Using cached einops-0.7.0-py3-none-any.whl (44 kB)
  Using cached transformers-4.35.0-py3-none-any.whl (7.9 MB)
  Using cached trimesh-4.0.5-py3-none-any.whl (688 kB)
  Using cached rembg-2.0.56-py3-none-any.whl (32 kB)
  Using cached imageio-2.34.1-py3-none-any.whl (313 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 12.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 KB 9.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.4 MB/s eta 0:00:00:00:0100:01
  Using cached tokenizers

In [13]:
# Import the TSR class
from tsr.system import TSR

Training Loop

In [14]:
import torch.nn as nn
import torch.optim as optim

# Assuming tripoSR is a class in your project, replace with actual import
from tsr.system import TSR 

# Load the pre-trained model
model = tsr(pretrained=True)

# Modify the model if necessary
# model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.MSELoss()  # Example, use appropriate loss for your task
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for vertices, faces in train_loader:
            vertices, faces = vertices.to(device), faces.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(vertices)
            loss = criterion(outputs, faces)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * vertices.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss}')
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for vertices, faces in val_loader:
                vertices, faces = vertices.to(device), faces.to(device)
                
                outputs = model(vertices)
                loss = criterion(outputs, faces)
                
                val_loss += loss.item() * vertices.size(0)
        
        val_loss = val_loss / len(val_loader.dataset)
        print(f'Validation Loss: {val_loss}')

# Train the model
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=25)

def evaluate(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for vertices, faces in test_loader:
            vertices, faces = vertices.to(device), faces.to(device)
            
            outputs = model(vertices)
            loss = criterion(outputs, faces)
            
            test_loss += loss.item() * vertices.size(0)
    
    test_loss = test_loss / len(test_loader.dataset)
    print(f'Test Loss: {test_loss}')

# Evaluate the model
evaluate(model, test_loader, criterion)

# Save the fine-tuned model
# torch.save(model.state_dict(), 'fine_tuned_tripoSR.pth')


NameError: name 'tsr' is not defined